# [OTA Dashboard](https://www.atmos.illinois.edu/~mgrover4/OT_map.html)

### Imports

In [1]:
import folium
from folium import plugins
import glob
import branca
import numpy as np
import geojson
from datetime import datetime
import branca.colormap as cm
from siphon.simplewebservice.spc import SPC
import pandas as pd

### Find lists of files from 10-11 August 2020 (Midwest Derecho) 

In [2]:
# Read files from the local system
aug_10_files = glob.glob('data/20200810*')
aug_11_files = glob.glob('data/20200811*')

In [3]:
# Merge into a single list of files
all_files = aug_10_files + aug_11_files

### Setup Functions to process data and create the map

In [4]:
# Define a dictionary for SPC reports where blue is wind, green is hail, and red is tornado
color_dict = {1:'blue',
              2:'green',
              3:'red'}

def grab_spc_data(date_range):
    """
    Reads in a pandas date range and creates a single dataframe with the reports,
    assigning identifiers for event type
    """
    
    # Create lists to store each hazard type
    wind_list = []
    tornado_list = []
    hail_list = []
    
    # Loop through the different days and pull SPC data
    for date in date_range:
        try:
            wind_list.append(SPC.get_wind_reports(date))
            tornado_list.append(SPC.get_tornado_reports(date))
            hail_list.append(SPC.get_hail_reports(date))
        except:
            None
    
    # Merge all the dataframes into a single dataframe for each hazard
    wind_df = pd.concat(wind_list)
    hail_df = pd.concat(hail_list)
    tornado_df = pd.concat(tornado_list)
    
    # Set integers to represent each hazard type
    wind_df['type'] = 1
    hail_df['type'] = 2
    tornado_df['type'] = 3
    
    # Merge all the hazards into a single dataframe
    all_hazards = pd.concat([wind_df, hail_df, tornado_df])
    
    # Ensure the time column is formatted as datetime and set as the index
    all_hazards.index = pd.to_datetime(all_hazards.Time)
    
    # Return the dataframe in chronological order
    return all_hazards.sort_index()

def data2geojson_ot(df):
    features = []
    max_ot = df.area_polygon.max()
    colormap = cm.linear.viridis.scale(0, int(max_ot))
    #colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], index=[0, int(max_ot/2), int(max_ot)], vmin=0,vmax=int(max_ot))
    colormap.caption = f'OT Area sq. km'
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["lon_corr"],
                                                    X["lat_corr"],)),
                            properties=dict(time=X["time"][:19],
                                            otarea=X["area_polygon"],
                                            icon='circle',
                                            radius=float(np.sqrt(X['area_polygon']/2)),
                                            style=dict(color=colormap(X['area_polygon']),
                                                       radius=float(np.sqrt(X['area_polygon']/2))),
                                            popup=f'<b>{X.time}</b> <br> <br> <b> OT Area: </b> {round(X.area_polygon,2)} sq. km <br> <br> <b> Minimum OT Temp: </b> {round(X.mintb, 2)} K <br> <br> <b> OT Prob </b>: {round(X.prob,2)} <br> <br> <b> OT Height: </b> {round(X.cloudtop_height, 2)} km')))
    df.apply(insert_features, axis=1)
    with open('test.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)
        
    return features



def plot_ot_data(files, output_file='ot_map.html', time_int=10):
    """
    Plots OT data from csv files
    """
    
    # Loop through the list of files and read into pandas dataframes
    df_list = []
    for file in files:
        try:
            df_list.append(pd.read_csv(file))
        except:
            None

    # Grab SPC data - using the filename to determine the date needed
    date_number = int(files[-1][-17:-9])
    
    try:
        spc_data = grab_spc_data(pd.date_range(start=str(date_number - 2), end=str(date_number+1)))

    except:
        spc_data = grab_spc_data(pd.date_range(start=str(date_number), end=str(date_number)))

    # Merge the files and drop the rating column
    df = pd.concat(df_list).drop(columns='Unnamed: 0')
    
    # Remove missing data
    df = df.dropna() 
    
    # Convert time to a datetime column
    df['time'] = pd.to_datetime(df.time)
    
    # Find the start and end time
    start_time = df.time.min()
    end_time = df.time.max()
    
    # Subset SPC data using the start and end time from the OT data
    spc_data = spc_data[(spc_data.index >= start_time) & (spc_data.index <= end_time)]
    
    # Parse time as a string
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M')
    df['time'] = df['time'].astype(str)
    
    # Find the maximum in OTA using the polygon area
    max_ot = df.area_polygon.max()
    
    # Setup a colormap ranging from 0 to the max OTA
    colormap = cm.linear.viridis.scale(0, int(max_ot))
    
    # Add a colormap title
    colormap.caption = f'OT Area (sq. km)'
    
    # Set the icon type to circle
    df['icon'] = 'circle'
    
    # Convert OT data to geojson
    features_ot = data2geojson_ot(df)
    
    # Create the map using folium
    OT_Map = folium.Map(location = [32, -85], zoom_start = 4, layout='vertical')
    
    # Add the OT layer
    ot_layer = plugins.TimestampedGeoJson({'type':'FeatureCollection',
                                           'features':features_ot}, period='PT15m', add_last_point=True,
                                            auto_play=True, loop_button=True, duration='PT1H', date_options='YYYY-MM-DD HH:mm').add_to(OT_Map)
    
    # Add the layer name which is displayed in the legend
    ot_layer.layer_name = 'OT Properties'
    
    # Create layers for each SPC hazard type
    hail_layer = folium.FeatureGroup(name="Hail Reports")
    wind_layer = folium.FeatureGroup(name="Wind Reports")
    tornado_layer = folium.FeatureGroup(name="Tornado Reports")
    
    # Shorten the name of the SPC dataframe
    spc = spc_data
    
    # Specify hazard type using the type column
    spc['haz_type'] = spc.type
    
    # Loop through and add circle markers for each SPC storm report
    for lat, lon, hazard, time, county, state, location, comments in zip(spc['Lat'], spc['Lon'], spc['haz_type'], spc['Time'], spc['County'], spc['State'], spc['Location'], spc['Comments']):
    
        point = folium.CircleMarker(
            [lat, lon],
            radius=3,
        
            popup = (f'<b>{time}</b> <br> <br>' +
                     f'<b> County: </b> {county} {state} <br> <br>' +
                     f'<b> Location </b>: {location} <br> <br>'+
                     f'<b> Comments: </b> {comments}'
                    ),
            color='b',
            key_on = hazard,
            fill_color=color_dict[hazard],
            fill=True,
            fill_opacity=0.7,
            control=True)
    
        if hazard == 1:
            wind_layer.add_child(point)
        
        elif hazard == 2:
            hail_layer.add_child(point)
        
        else:
            tornado_layer.add_child(point)
        
    # Add the SPC layers to the map
    OT_Map.add_child(wind_layer)
    OT_Map.add_child(hail_layer)
    OT_Map.add_child(tornado_layer)
    
    # Enable a fullscreen option
    folium.plugins.Fullscreen(position='topleft', title='Expand me', title_cancel='Exit me', force_separate_button=True).add_to(OT_Map)
    
    # Add the colormap to the map
    OT_Map.add_child(colormap)
    
    # Enable the user to control the layers
    folium.LayerControl().add_to(OT_Map)
    
    # Add a measuring tool
    OT_Map.add_child(folium.plugins.MeasureControl())

    # Add titles and footers
    title_html = f'''
                 <h3 align="left" style="font-size:18px">Valid: {df.time.values[-1]} UTC</h3>
                 '''

    footer_html=f'''
                <h5 align="left" style="font-size:12px">Data Courtesy of Kris Bedka and NOAA</h5>
                '''

    third_html = f'''
                <h5 align="left" style="font-size:12px">Valid: {df.time.values[-1]}</h5>
                '''

    main_title_html = '''
                     <h2 align="left" style="font-size:24px"><b> History of Overshooting Top Area (OTA) </b></h2>
                     '''
    
    # Add titles/footers to the page
    OT_Map.get_root().html.add_child(folium.Element(main_title_html))
    OT_Map.get_root().html.add_child(folium.Element(title_html))
    OT_Map.add_child(colormap)
    OT_Map.get_root().html.add_child(folium.Element(footer_html))
    
    # Save the map
    OT_Map.save(output_file)
    
    # Return the map
    return OT_Map

In [5]:
plot_ot_data(all_files)